# ETL
This notebook loads in the dataset, enforces correct typecasting, adds a column containing all of the duplicates for each row, and saves the dataset as a pickle file.

In [1]:
import pandas as pd

In [2]:
import os
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /home/msa242/CMPT413-Final-Project/ETL


### Read in Data

In [3]:
dataset_name = 'mozilla_firefox'
df = pd.read_csv('../data/' + dataset_name + '.csv')
df.head()

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,10954,P3,Preferences,NaN,Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700
1,14871,--,General,269442.0,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700
2,19118,--,Preferences,NaN,Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800
3,54746,P3,Preferences,NaN,Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800
4,56892,P3,General,NaN,Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700


In [4]:
df.shape

(115814, 11)

### Typecast

In [5]:
def typecast_df(df):
    df['Content'] = df['Title'] + ' ' + df['Description']
    df['Duplicated_issue'] = pd.to_numeric(df['Duplicated_issue'], errors='coerce').astype('Int64')
    df["Issue_id"] = df["Issue_id"].astype('Int64')
    return df
df = typecast_df(df)

In [6]:
df.head()

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Content
0,10954,P3,Preferences,<NA>,Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700,Dialup properties needs to be exposed in prefs...
1,14871,--,General,269442,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700,[Find] Find whole word only Please add Match W...
2,19118,--,Preferences,<NA>,Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800,Plug-In Manager (ui for choosing mimetype-plug...
3,54746,P3,Preferences,<NA>,Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800,Language encodings in font prefs dialog not so...
4,56892,P3,General,<NA>,Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700,Synaptics touchpad scrolling not working From ...


### Find Duplicated_issue ids that do not have corresponding entries

In [7]:
missing_issues = df[~df['Duplicated_issue'].isna() & ~df['Duplicated_issue'].isin(df['Issue_id'])]
missing_issues

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Content
1,14871,--,General,269442,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700,[Find] Find whole word only Please add Match W...
64,167422,--,General,162593,view source tries to download type application...,view source treis to download application x-vi...,VERIFIED,DUPLICATE,unspecified,2002-09-08 18:06:09 -0700,2002-09-19 13:24:25 -0700,view source tries to download type application...
92,169727,--,General,126730,Restored window position and size lost after a...,Tested on: Mozilla/5.0 (X11; U; Linux i686; en...,RESOLVED,DUPLICATE,Trunk,2002-09-19 13:01:39 -0700,2008-04-06 11:55:18 -0700,Restored window position and size lost after a...
101,170029,--,Menus,25894,Respect Windows Access Keys display pref (defa...,only shows the access keys when you hit alt,RESOLVED,DUPLICATE,unspecified,2002-09-20 23:20:38 -0700,2009-03-30 13:54:45 -0700,Respect Windows Access Keys display pref (defa...
106,170216,--,General,280190,Pretty up dns error page a bit,,VERIFIED,DUPLICATE,unspecified,2002-09-22 14:24:45 -0700,2012-06-04 19:34:48 -0700,Pretty up dns error page a bit
...,...,...,...,...,...,...,...,...,...,...,...,...
115785,953147,--,Security,616853,Ransomware locks Firefox tab; uses onbeforeunl...,Created attachment 8351469; HTML from the site...,RESOLVED,DUPLICATE,Trunk,2013-12-25 21:14:32 -0800,2013-12-26 17:48:18 -0800,Ransomware locks Firefox tab; uses onbeforeunl...
115790,953223,--,Menus,787875,right clicking on an image should offer Google...,Created attachment 8351530; f.png; ; User Agen...,RESOLVED,DUPLICATE,28 Branch,2013-12-26 16:58:37 -0800,2013-12-27 10:49:27 -0800,right clicking on an image should offer Google...
115795,953332,--,Untriaged,248955,Per-tab search engine search bar,User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64...,RESOLVED,DUPLICATE,26 Branch,2013-12-28 03:24:38 -0800,2013-12-28 06:08:41 -0800,Per-tab search engine search bar User Agent: M...
115801,953436,--,Untriaged,50673,partial copying of hyperlinked text,User Agent: Mozilla/5.0 (X11; Linux i686; rv:2...,RESOLVED,DUPLICATE,28 Branch,2013-12-29 16:34:33 -0800,2013-12-30 00:21:36 -0800,partial copying of hyperlinked text User Agent...


### Create Duplicated_issue column containing an array of Duplicated_issue ids

In [8]:
duplicates_dict = df.groupby('Issue_id')['Duplicated_issue'].apply(set).to_dict()

def get_all_duplicates(issue_id):
    duplicates = set()
    issues_to_check = [issue_id]

    while issues_to_check:
        current_issue = issues_to_check.pop()
        current_duplicates = duplicates_dict.get(current_issue, set())

        new_duplicates = current_duplicates - duplicates
        duplicates.update(new_duplicates)
        issues_to_check.extend(new_duplicates)

    return list(duplicates) if duplicates else []

df['Duplicated_issues'] = df['Issue_id'].apply(get_all_duplicates)

In [9]:
def clean_convert_array(arr):
    cleaned = [int(x) for x in arr if not pd.isnull(x)]
    return cleaned

df['Duplicated_issues'] = df['Duplicated_issues'].apply(clean_convert_array)
df = df.sort_values(by='Duplicated_issues', key=lambda x: x.str.len(), ascending=False)

In [10]:
df.head()

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Content,Duplicated_issues
33591,335190,--,General,335186,Ctrl+C does not copy,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2006-04-23 18:45:42 -0700,2006-04-23 19:50:22 -0700,Ctrl+C does not copy User-Agent: Mozilla...,"[335186, 334500, 334862]"
21703,294616,--,Menus,236336,Open URL in context menu,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2005-05-18 02:59:14 -0700,2006-01-19 00:20:44 -0800,Open URL in context menu User-Agent: Moz...,"[236336, 227922, 454518]"
104453,787029,--,Untriaged,787022,movement was canceled in web page Navigation...,User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64...,RESOLVED,DUPLICATE,14 Branch,2012-08-30 06:12:03 -0700,2012-08-30 06:30:58 -0700,movement was canceled in web page Navigation...,"[651803, 787021, 787022]"
1900,203901,--,Toolbars and Customization,203899,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:21:59 -0700,2006-11-13 07:16:57 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]"
1898,203898,--,Toolbars and Customization,203899,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:10:53 -0700,2006-11-13 07:23:00 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]"


### Count the number of issues with duplicates before removing nonexistent id's

In [11]:
num_issues_with_duplicates = df['Duplicated_issues'].apply(lambda x: len(x) > 0).sum()
num_issues_with_duplicates

35814

### Remove the nonexistent id's from Duplicate_issues arrays

In [12]:
issue_ids = set(df['Issue_id'])

def remove_nonexistent_issue_ids(duplicated_issues):
    return [issue_id for issue_id in duplicated_issues if issue_id in issue_ids]

df['Duplicated_issues'] = df['Duplicated_issues'].apply(remove_nonexistent_issue_ids)
df.head()

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Content,Duplicated_issues
33591,335190,--,General,335186,Ctrl+C does not copy,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2006-04-23 18:45:42 -0700,2006-04-23 19:50:22 -0700,Ctrl+C does not copy User-Agent: Mozilla...,"[335186, 334862]"
21703,294616,--,Menus,236336,Open URL in context menu,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2005-05-18 02:59:14 -0700,2006-01-19 00:20:44 -0800,Open URL in context menu User-Agent: Moz...,"[236336, 227922, 454518]"
104453,787029,--,Untriaged,787022,movement was canceled in web page Navigation...,User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64...,RESOLVED,DUPLICATE,14 Branch,2012-08-30 06:12:03 -0700,2012-08-30 06:30:58 -0700,movement was canceled in web page Navigation...,"[651803, 787021, 787022]"
1900,203901,--,Toolbars and Customization,203899,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:21:59 -0700,2006-11-13 07:16:57 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]"
1898,203898,--,Toolbars and Customization,203899,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:10:53 -0700,2006-11-13 07:23:00 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]"


In [13]:
num_issues_with_duplicates = df['Duplicated_issues'].apply(lambda x: len(x) > 0).sum()
num_issues_with_duplicates

15742

After removing the non-existent issue ids, we reduce the number of issues with duplicates from $35814$ to $15742$, a reduction by $56\%$

### Clean up
- Remove the "Duplicated_issue" column
- Set "Resolution" with no duplicates and Resolution==DUPLICATE to Resolution==NDUPLICATE
- Add "Duplicates_count" column

In [14]:
df = df.drop(columns=['Duplicated_issue'])
df.loc[(df['Duplicated_issues'].str.len() == 0) & (df['Resolution'] == 'DUPLICATE'), 'Resolution'] = 'NDUPLICATE'
df['Duplicates_count'] = df['Duplicated_issues'].apply(len)
df.head()

,Issue_id,Priority,Component,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Content,Duplicated_issues,Duplicates_count
33591,335190,--,General,Ctrl+C does not copy,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2006-04-23 18:45:42 -0700,2006-04-23 19:50:22 -0700,Ctrl+C does not copy User-Agent: Mozilla...,"[335186, 334862]",2
21703,294616,--,Menus,Open URL in context menu,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2005-05-18 02:59:14 -0700,2006-01-19 00:20:44 -0800,Open URL in context menu User-Agent: Moz...,"[236336, 227922, 454518]",3
104453,787029,--,Untriaged,movement was canceled in web page Navigation...,User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64...,RESOLVED,DUPLICATE,14 Branch,2012-08-30 06:12:03 -0700,2012-08-30 06:30:58 -0700,movement was canceled in web page Navigation...,"[651803, 787021, 787022]",3
1900,203901,--,Toolbars and Customization,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:21:59 -0700,2006-11-13 07:16:57 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]",3
1898,203898,--,Toolbars and Customization,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:10:53 -0700,2006-11-13 07:23:00 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]",3


### Save as Pickle File

In [15]:
df.to_pickle('../data/' + dataset_name + '.pkl')

### Read the pickle file into a dataframe

In [16]:
df = pd.read_pickle('../data/' + dataset_name + '.pkl')
df.head()

,Issue_id,Priority,Component,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Content,Duplicated_issues,Duplicates_count
33591,335190,--,General,Ctrl+C does not copy,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2006-04-23 18:45:42 -0700,2006-04-23 19:50:22 -0700,Ctrl+C does not copy User-Agent: Mozilla...,"[335186, 334862]",2
21703,294616,--,Menus,Open URL in context menu,User-Agent: Mozilla/5.0 (Windows; U; Win...,RESOLVED,DUPLICATE,unspecified,2005-05-18 02:59:14 -0700,2006-01-19 00:20:44 -0800,Open URL in context menu User-Agent: Moz...,"[236336, 227922, 454518]",3
104453,787029,--,Untriaged,movement was canceled in web page Navigation...,User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64...,RESOLVED,DUPLICATE,14 Branch,2012-08-30 06:12:03 -0700,2012-08-30 06:30:58 -0700,movement was canceled in web page Navigation...,"[651803, 787021, 787022]",3
1900,203901,--,Toolbars and Customization,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:21:59 -0700,2006-11-13 07:16:57 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]",3
1898,203898,--,Toolbars and Customization,Folders in Bookmarks Toolbar stay Expanded if ...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,DUPLICATE,unspecified,2003-04-30 03:10:53 -0700,2006-11-13 07:23:00 -0800,Folders in Bookmarks Toolbar stay Expanded if ...,"[197227, 203899, 210910]",3
